## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
#from google.colab import drive
#drive.mount('/gdrive')

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('prices.csv')

### Check all columns in the dataset

In [6]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [8]:
data.drop("date", axis=1, inplace=True)
data.drop("symbol", axis=1, inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
df = data.head(1000)

In [11]:
X = df.drop("volume", axis=1)
Y = df["volume"] 

In [12]:
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [13]:
Y.head()

0    2163600.0
1    2386400.0
2    2489500.0
3    2006300.0
4    1408600.0
Name: volume, dtype: float64

In [14]:
Y = Y.divide(1000000)

In [15]:
Y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [16]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.3,random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [17]:
import numpy as np
x_train = np.array(xtrain, dtype="float32")
y_train = np.array(ytrain, dtype="float32")
x_test = np.array(xtest, dtype="float32")
y_test = np.array(ytest, dtype="float32")

In [18]:
x_train

array([[ 27.77   ,  27.65   ,  27.41   ,  27.84   ],
       [ 12.94   ,  13.33   ,  12.88   ,  13.37   ],
       [ 18.57   ,  18.89   ,  18.4    ,  18.97   ],
       ...,
       [627.18   , 623.99   , 621.54004, 627.83997],
       [125.19   , 123.     , 121.95   , 125.91   ],
       [113.04   , 113.32   , 111.65   , 115.36   ]], dtype=float32)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [19]:
from sklearn.preprocessing import Normalizer
N = Normalizer()
x_train=N.fit_transform(x_train)
x_test=N.transform(x_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [20]:
W = tf.Variable(tf.random_normal(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
W

<tf.Variable 'Weights:0' shape=(4, 1) dtype=float32_ref>

2.Define a function to calculate prediction

In [22]:
y_pred = tf.add(tf.matmul(x_train,W),b,name='output')

3.Loss (Cost) Function [Mean square error]

In [23]:
loss = tf.reduce_mean(tf.square(y_pred-y_train),name='mse')

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [24]:
train_gd = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [25]:
x = tf.placeholder(shape=[None,4], dtype=tf.float32)  # x place holder
y = tf.placeholder(shape=[None], dtype=tf.float32) # y place holder

In [26]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
for epoch in range(training_epochs):
    _, train_loss = sess.run([train_gd,loss],feed_dict={x:x_train, y:y_train})  
    print ('Training loss at every step: ', epoch, ' is ', train_loss)

Training loss at every step:  0  is  232.58478
Training loss at every step:  1  is  224.19688
Training loss at every step:  2  is  217.70125
Training loss at every step:  3  is  212.67102
Training loss at every step:  4  is  208.77553
Training loss at every step:  5  is  205.7589
Training loss at every step:  6  is  203.42279
Training loss at every step:  7  is  201.61365
Training loss at every step:  8  is  200.21269
Training loss at every step:  9  is  199.12776
Training loss at every step:  10  is  198.2876
Training loss at every step:  11  is  197.63696
Training loss at every step:  12  is  197.13313
Training loss at every step:  13  is  196.74292
Training loss at every step:  14  is  196.44075
Training loss at every step:  15  is  196.20674
Training loss at every step:  16  is  196.02559
Training loss at every step:  17  is  195.88524
Training loss at every step:  18  is  195.77658
Training loss at every step:  19  is  195.69238
Training loss at every step:  20  is  195.62726
Trai

In [27]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
training_epochs = 100
for epoch in range(training_epochs):
    _, train_loss = sess.run([train_gd,loss],feed_dict={x:x_train, y:y_train})  
    if epoch % 5 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  227.21277
Training loss at step:  5  is  204.26273
Training loss at step:  10  is  197.8709
Training loss at step:  15  is  196.09073
Training loss at step:  20  is  195.59491
Training loss at step:  25  is  195.45682
Training loss at step:  30  is  195.41838
Training loss at step:  35  is  195.40764
Training loss at step:  40  is  195.40468
Training loss at step:  45  is  195.40387
Training loss at step:  50  is  195.40363
Training loss at step:  55  is  195.40353
Training loss at step:  60  is  195.40353
Training loss at step:  65  is  195.40349
Training loss at step:  70  is  195.4035
Training loss at step:  75  is  195.40353
Training loss at step:  80  is  195.40353
Training loss at step:  85  is  195.40353
Training loss at step:  90  is  195.40353
Training loss at step:  95  is  195.40353


### Get the shapes and values of W and b

In [28]:
sess.run(W)

array([[1.7653917],
       [0.9126457],
       [0.876212 ],
       [1.8377709]], dtype=float32)

In [29]:
sess.run(b)

array([2.8200471], dtype=float32)

### Model Prediction on 1st Examples in Test Dataset

In [30]:
y_test_pred = tf.add(tf.matmul(x_test,W),b,name='output')

In [31]:
sess.run(y_test_pred)

array([[5.52069  ],
       [5.5178475],
       [5.5181646],
       [5.5187426],
       [5.517969 ],
       [5.5172753],
       [5.516909 ],
       [5.520095 ],
       [5.52391  ],
       [5.5191813],
       [5.518934 ],
       [5.518276 ],
       [5.518308 ],
       [5.5182056],
       [5.5279427],
       [5.523489 ],
       [5.519987 ],
       [5.5317373],
       [5.5210533],
       [5.5263186],
       [5.5185623],
       [5.5186005],
       [5.518197 ],
       [5.522723 ],
       [5.517437 ],
       [5.533416 ],
       [5.517453 ],
       [5.5202055],
       [5.5232716],
       [5.5252967],
       [5.518528 ],
       [5.5212584],
       [5.526249 ],
       [5.519371 ],
       [5.527028 ],
       [5.5175714],
       [5.5193944],
       [5.518876 ],
       [5.5203047],
       [5.520211 ],
       [5.5205383],
       [5.5193815],
       [5.5244055],
       [5.519621 ],
       [5.5202456],
       [5.5225983],
       [5.5184803],
       [5.5188503],
       [5.523817 ],
       [5.5166206],


In [32]:
sess.close()

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [33]:
iris = pd.read_csv('Iris.csv')
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [34]:
iris.drop("Id", axis=1, inplace=True)

In [35]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [36]:
iris['Species'].value_counts()

Iris-versicolor    50
Iris-setosa        50
Iris-virginica     50
Name: Species, dtype: int64

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [37]:
# pd.get_dummies(iris, columns=['Species'], drop_first = True)
dummy = pd.get_dummies(iris['Species'])
dummy.sample(10)

,Iris-setosa,Iris-versicolor,Iris-virginica
40,1,0,0
88,0,1,0
19,1,0,0
42,1,0,0
21,1,0,0
14,1,0,0
51,0,1,0
15,1,0,0
11,1,0,0
103,0,0,1


### Splitting the data into feature set and target set

In [38]:
X = iris.drop("Species", axis=1)
Y = dummy

In [39]:
X.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [40]:
Y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [41]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=4, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

Using TensorFlow backend.


### Model Training 

In [42]:
model.fit(X, Y, epochs=100, batch_size=5) 

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
150/150 [==============================] - 1s 6ms/step - loss: 0.9676 - acc: 0.6400
Epoch 2/100
150/150 [==============================] - 0s 1ms/step - loss: 0.6705 - acc: 0.8133
Epoch 3/100
150/150 [==============================] - 0s 667us/step - loss: 0.5757 - acc: 0.8800
Epoch 4/100
150/150 [==============================] - 0s 767us/step - loss: 0.5368 - acc: 0.8667
Epoch 5/100
150/150 [==============================] - 0s 683us/step - loss: 0.4831 - acc: 0.8400
Epoch 6/100
150/150 [==============================] - 0s 717us/step - loss: 0.4642 - acc: 0.8333
Epoch 7/100
150/150 [==============================] - 0s 650us/step - loss: 0.4267 - acc: 0.8933
Epoch 8/100
150/150 [==============================] - 0s 983us/step - loss: 0.3941 - acc: 0.8867
Epoch 9/100
150/150 [==============================] - 0s 600us/step - loss: 0.3965 - acc: 0.8600
Epoch 10/100
150/150 [==============================] - 0s 650us/step - lo

150/150 [==============================] - 0s 467us/step - loss: 0.1177 - acc: 0.9667
Epoch 82/100
150/150 [==============================] - 0s 450us/step - loss: 0.1133 - acc: 0.9667
Epoch 83/100
150/150 [==============================] - 0s 567us/step - loss: 0.1150 - acc: 0.9667
Epoch 84/100
150/150 [==============================] - 0s 850us/step - loss: 0.1133 - acc: 0.9600
Epoch 85/100
150/150 [==============================] - 0s 567us/step - loss: 0.1010 - acc: 0.9800
Epoch 86/100
150/150 [==============================] - 0s 517us/step - loss: 0.1202 - acc: 0.9600
Epoch 87/100
150/150 [==============================] - 0s 467us/step - loss: 0.1217 - acc: 0.9533
Epoch 88/100
150/150 [==============================] - 0s 533us/step - loss: 0.1095 - acc: 0.9600
Epoch 89/100
150/150 [==============================] - 0s 567us/step - loss: 0.1201 - acc: 0.9467
Epoch 90/100
150/150 [==============================] - 0s 467us/step - loss: 0.0989 - acc: 0.9733
Epoch 91/100
150/150 [=

### Model Prediction

In [43]:
scores = model.evaluate(X, Y)
scores[1]*100

150/150 [==============================] - 0s 1ms/step


96.66666666666667

In [44]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=5)

In [45]:
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=100)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 262us/step - loss: 0.0839 - acc: 0.9810 - val_loss: 0.0939 - val_acc: 0.9556
Epoch 2/100
105/105 [==============================] - 0s 190us/step - loss: 0.0831 - acc: 0.9905 - val_loss: 0.1067 - val_acc: 0.9333
Epoch 3/100
105/105 [==============================] - 0s 190us/step - loss: 0.0858 - acc: 0.9810 - val_loss: 0.1066 - val_acc: 0.9333
Epoch 4/100
105/105 [==============================] - 0s 143us/step - loss: 0.0823 - acc: 0.9810 - val_loss: 0.0964 - val_acc: 0.9556
Epoch 5/100
105/105 [==============================] - 0s 333us/step - loss: 0.0808 - acc: 0.9905 - val_loss: 0.0815 - val_acc: 0.9778
Epoch 6/100
105/105 [==============================] - 0s 167us/step - loss: 0.0823 - acc: 0.9905 - val_loss: 0.1081 - val_acc: 0.9333
Epoch 7/100
105/105 [==============================] - 0s 167us/step - loss: 0.0863 - acc: 0.9810 - val_loss: 0.0936 - val_acc: 0.9556
Epoch 8/10

Epoch 61/100
105/105 [==============================] - 0s 238us/step - loss: 0.0788 - acc: 0.9810 - val_loss: 0.1042 - val_acc: 0.9333
Epoch 62/100
105/105 [==============================] - 0s 143us/step - loss: 0.0797 - acc: 0.9810 - val_loss: 0.1093 - val_acc: 0.9333
Epoch 63/100
105/105 [==============================] - 0s 143us/step - loss: 0.0792 - acc: 0.9810 - val_loss: 0.1019 - val_acc: 0.9556
Epoch 64/100
105/105 [==============================] - ETA: 0s - loss: 0.0573 - acc: 1.000 - 0s 214us/step - loss: 0.0794 - acc: 0.9810 - val_loss: 0.0972 - val_acc: 0.9556
Epoch 65/100
105/105 [==============================] - 0s 214us/step - loss: 0.0788 - acc: 0.9905 - val_loss: 0.1260 - val_acc: 0.9333
Epoch 66/100
105/105 [==============================] - 0s 167us/step - loss: 0.0835 - acc: 0.9810 - val_loss: 0.0791 - val_acc: 0.9778
Epoch 67/100
105/105 [==============================] - 0s 143us/step - loss: 0.0823 - acc: 0.9905 - val_loss: 0.0955 - val_acc: 0.9556
Epoch 68/1

In [49]:
y_predict = model.predict(x_test)

In [55]:
y_predict

array([[2.92037753e-03, 9.84696507e-01, 1.23830820e-02],
       [4.01187935e-06, 3.72198899e-03, 9.96273994e-01],
       [9.26102700e-07, 2.03203468e-04, 9.99795854e-01],
       [9.95187402e-01, 4.77912743e-03, 3.34263095e-05],
       [9.96651124e-07, 1.10788026e-03, 9.98891175e-01],
       [9.44462791e-03, 9.73113060e-01, 1.74424220e-02],
       [9.94576454e-01, 5.40219154e-03, 2.12948507e-05],
       [3.60415695e-04, 1.50545254e-01, 8.49094331e-01],
       [9.91444170e-01, 8.52759648e-03, 2.82821147e-05],
       [8.85488465e-04, 2.84137815e-01, 7.14976668e-01],
       [1.19955011e-03, 4.49741691e-01, 5.49058735e-01],
       [1.70757805e-04, 1.38952687e-01, 8.60876620e-01],
       [1.94180916e-06, 2.40201550e-03, 9.97596085e-01],
       [7.68788311e-07, 1.77429954e-03, 9.98224914e-01],
       [9.95567262e-01, 4.41867486e-03, 1.39732319e-05],
       [9.91286814e-01, 8.68441444e-03, 2.88216888e-05],
       [1.21828598e-04, 6.50773272e-02, 9.34800804e-01],
       [1.61063360e-06, 3.23704

### Save the Model

In [ ]:
import pickle
pickle.dump(model, open("model.pkl","wb" , ))
#model = pickle.load(open("model.pkl", "rb"))
##"rb" : this means read mode

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?